In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


The process of reading the file is exactly the same as for the case of W boson at rest. Only difference is the number of branches used for the four-vectors.

In [2]:
file = r.TFile("generatedComponentsBOOST.root","READ")
tree = file.Get("tree") # Getting the tree 

Warning in <TFile::Init>: file generatedComponentsBOOST.root probably not closed, trying to recover
Info in <TFile::Recover>: generatedComponentsBOOST.root, recovered key TTree:tree at address 96692
Warning in <TFile::Init>: successfully recovered 1 keys


In [3]:
# Initializing 4-element array to store 4-vectors
wArr = np.zeros(4)
eArr = np.zeros(4)
betaArr = np.zeros(4)
nArr = np.zeros(4)

In [4]:
# Taking already filled branches from the file
# Now we need to add another branch for neutrino
tree.SetBranchAddress("wBoson", wArr)
tree.SetBranchAddress("electron", eArr)
tree.SetBranchAddress("beta", betaArr)

# Initializing branch for neutrino components
tree.Branch("neutrino",nArr,"neArray[4]/D")

# Minimisation

Works in the same way as the minimizer for **stationary W**. 

What is changed is the function for W boson mass. The loss function is still the same.
The choice of initial value is different, as we have to make an initial assumption, such that the TMinuit would return
the value with approprite sign.


In [5]:
%%cpp -d   

void fcn(int &npar, double *gin, double &f, double *par, int iflag){
    double pTSq = par[0];
    double pTotSq = par[1];
    double ePz = par[2];
    double nPz = par[3];

     /* calculating mass of W boson as a funtion of anti neutrino Pz */
    double wMass = TMath::Sqrt(2*pTSq + 2 * TMath::Sqrt(pTotSq) * TMath::Sqrt(pTSq + pow(nPz,2)) - 2 * ePz * nPz);

    f = pow(wMass-80.3,2); /* loss function to be minimised */
};

double minimization_func(double *wArr, double *elArr){
    /*  Input:
        wArr ...... W boson four-vector as an 4-element array [Px,Py,Pz,E]
        elArr...... Electron four-vector as an 4-element array [Px,Py,Pz,E]
        
        Output:
        nPz ....... float value of anti neutrino Pz
    */ 
    
    double ePx = elArr[0];
    double ePy = elArr[1];
    double ePz = elArr[2];
    
    double wPz = wArr[2]; 
    
    double initVal;
    
    if(abs(ePz - wPz) > 1){
        initVal = -(ePz - wPz)+1;
    } else {
        initVal = -ePz + 0.02;  
    }
    
    double pTSqEl = pow(ePx,2) + pow(ePy,2);
    double pTotSqEl = pTSqEl + pow(ePz,2);  
    
    TMinuit *gMinuit = new TMinuit(4);
    gMinuit->SetFCN(fcn);
    gMinuit->DefineParameter(0, "pTotSqEl", pTSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(1, "pTSqEl", pTotSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(2, "ePz", ePz, 1., 0., 0.);
    gMinuit->DefineParameter(3, "nPz", initVal, 0.05, 0., 0.);
    gMinuit->FixParameter(0);
    gMinuit->FixParameter(1);
    gMinuit->FixParameter(2);
    gMinuit->Command("MIGRAD");

    double nPz, nPzErr;
    gMinuit->GetParameter(3, nPz, nPzErr);
    return nPz;

};

Code to fill branches in TTree with their respective values.

In [6]:
n = tree.GetEntries() # Number of entries in tree (number of different events)

for i in range(n):
    tree.GetEntry(i)
    ePx = tree.GetListOfLeaves().At(1).GetValue(0)
    ePy = tree.GetListOfLeaves().At(1).GetValue(1)
    
    nArr[0] = -ePx
    nArr[1] = -ePy
    nArr[2] = r.minimization_func(wArr, eArr)
    nArr[3] = r.TMath.Sqrt(nArr[0]**2 + nArr[1]**2 + nArr[2]**2)
    
    tree.GetBranch("neutrino").Fill() # Filling just "neutrino" branch

 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 pTotSqEl     1.55587e+03  1.00000e+00     no limits
     2 pTSqEl       1.74463e+03  1.00000e+00     no limits
     3 ePz         -1.37389e+01  1.00000e+00     no limits
     4 nPz          2.42306e+00  5.00000e-02     no limits
 **********
 **    1 **FIX           1
 **********
 **********
 **    2 **FIX           2
 **********
 **********
 **    3 **FIX           3
 **********
 **********
 **    4 **MIGRAD 
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-04
 FCN=0.0384877 FROM MIGRAD    STATUS=INITIATE        4 CALLS           5 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
  EXT PARAMETER               CURRENT GUESS       STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  pTotSqEl     1.55587e+03     fixed    


Error in <TBranch::TBranch::WriteBasketImpl>: basket's WriteBuffer failed.

Error in <TBranch::TBranch::Fill>: Failed to write out basket.



Scanning the TTree to see its structure.

In [7]:
tree.Scan("wBoson:electron:neutrino:beta")

4000

***********************************************************************
*    Row   * Instance *    wBoson *  electron *  neutrino *      beta *
***********************************************************************
*        0 *        0 *         0 * -37.89209 *  37.89209 * -0.151600 *
*        0 *        1 *         0 * 10.957255 * -10.95725 *         0 *
*        0 *        2 * -12.31585 * -13.73891 * 1.4231111 *         0 *
*        0 *        3 * 81.238970 * 41.768765 * 39.470206 *         0 *
*        1 *        0 *         0 * -35.48510 * 35.485101 * 0.1097648 *
*        1 *        1 *         0 * 15.600797 * -15.60079 *         0 *
*        1 *        2 * 8.8676978 * 14.959051 * -6.091362 *         0 *
*        1 *        3 * 80.788155 * 41.549374 * 39.238782 *         0 *
*        2 *        0 *         0 * 25.936019 * -25.93601 * -0.079983 *
*        2 *        1 *         0 * -25.34654 * 25.346545 *         0 *
*        2 *        2 * -6.443283 * 14.064469 * -20.50707 *     

Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or 